# 인공신경망
## 퍼셉트론
- 인공신경망의 한 종류
- 가장 간단한 형태의 피드포워드 네트워크 (선형 분류기)

# Online Learning Perceptron in Python
파이썬으로 표준 라이브러리를 사용해서 위의 퍼셉트론 알고리즘을 구현했기 때문에 스크립트가 PyPy에서 실행되고 3-4배의 속도향상이 있다. 여기에 사용된 알고리즘은 Kaggle 포럼에서 처음 발견 된 tinrtgu의 온라인 로지스틱 회귀 스크립트에서 큰 영감을 얻었다고 한다.
- 다음 경진대회에서 Vowpal Wabbit을 통한 해시트릭을 사용하였고 코드가 공개되어 있다.
- Display Advertising Challenge - Kaggle, Beat the benchmark with less then 200MB of memory.
- 코드 : https://kaggle2.blob.core.windows.net/forum-message-attachments/53646/1539/fast_solution.py

# 온라인 학습
퍼셉트론은 온라인 학습이 가능하다(한 번에 하나씩 샘플을 통해 학습). 메모리에 전체 데이터 세트가 필요하지 않으므로(out-of-core) 더 큰 데이터 세트에 유용하다. 여기에서는 Vowpal Wabbit에서 온라인 학습 코드를 가져왔다.
- Vowpal Wabbit (Fast Learning) : http://hunch.net/~vw/
- Hashing Representations : http://hunch.net/~jl/projects/hash_reps/

# 해싱 트릭
#### 메모리를 적게 사용할 수 있다
벡터화 해싱 트릭은 Vowpal Wabbit(John Langford)에서 시작되었다. 이 트릭은 퍼셉트론으로 들어오는 연결 수를 고정 된 크기로 설정한다. 고정 된 크기보다 낮은 숫자로 모든 원시 피처를 해싱한다. Vowpal Wabbit은 모든 데이터를 메모리로 읽어들이지 않고 모델을 훈련 시킬 수 있는 빠른 머신러닝 라이브러리다.

In [1]:
sample = "This movie sucks"
fixed_size = 1024

print(sample.split())

features = [(hash(f)%fixed_size, 1) for f in sample.split()]

print(features)

['This', 'movie', 'sucks']
[(705, 1), (335, 1), (450, 1)]


# 프로그레시브 검증 손실
- 한 번에 하나씩 표본을 학습하면 점진적으로 train loss가 된다. 모델이 타겟을 보지않고 첫 샘플을 보고 예측을 한다.
- 그런 다음 예측을 대상 레이블과 비교하여 오류율을 계산 한다. 오류율이 낮으면 좋은 모델에 가깝다.

# Multiple passes
- Vowpal Wabbit을 사용하면 학습률이 떨어지는 데이터 집합을 여러 번 통과시킬 수 있다.
- 오류율이 낮아지면 데이터 세트를 여러 번 실행할 수도 있다. 트레이닝 데이터가 선형 적으로 분리 가능한 경우, 퍼셉트론은 트레이닝 세트에서 오차가 0으로 수렴된다.

In [2]:
# opts["D"] 여기에 있는 코드에서는 고정 값으로 다음의 값을 사용한다.
2 ** 25

33554432

In [4]:
import re # 정규표현식으로 텍스트 데이터를 정제
import random
from math import exp, log # 지수, 로그 함수 사용
from datetime import datetime # 시간 계산
from operator import itemgetter # 키가 아닌 값으로 max, min 값을 구할 때 사용

In [15]:
def clean(s):
    '''
    텍스트 정제, 소문자로 변환
    '''
    return " ".join(re.findall(r'\w+', s, flags = re.UNICODE)).lower()

In [22]:
def get_data_tsv(loc_dataset, opts):
    '''
    온라인 학습 방법을 통해 데이터를 실행한다.
    tsv파일을 통해 레이블, identifier, 피처(특성)를 파싱한다.
    결과물:
        label : int, 레이블 / 대상 (테스트 집합 인 경우 "1"로 설정)
        id : 문자열, 샘플 식별자
        features : [(hashed_feature_index, feature_value)] 
                형식의 튜플 목록    
    '''
    for e, line in enumerate(open(loc_dataset, "rb")):
        if e > 0:
            r = line.decode('utf-8').strip().split('\t')
            id = r[0]
            
            if opts['clean']:
                try:
                    r[2] = clean(r[2]) # train
                except:
                    r[1] = clean(r[1]) # test
            
            # opts["D"] = 2 ** 25 = 33554432
            # Vowpal Wabbit의 해싱트릭을 사용한다.
            # 해싱트릭은 큰 규모의 feature공간을 고정크기의 표현을 사용해 저장할 수 있게 한다.
            
            if len(r) == 3: # train set
                features = [(hash(f)%opts["D"],1) for f in r[2].split()]
                label = int(r[1])
            else: # test set
                features = [(hash(f)%opts["D"],1) for f in r[1].split()]
                label = 1
                
            # bigram을 사용하면 해당 피처[i]와 다음피처[i+1]를 함께 해싱한다.
            if opts["2grams"]:
                for i in range(len(features)-1):
                    features.append(
                        (hash(str(features[i][0])+str(features[i+1][0]))%opts["D"],1))
            yield label, id, features

In [17]:
def dot_product(features, weights):
    """
    피처(특성)과 가중치로부터 내적을 구한다.
    """
    dotp = 0
    for f in features:
        dotp += weights[f[0]] * f[1]
        
    return dotp    

In [18]:
def train_tron(loc_dataset, opts):
    start = datetime.now()
    print("\nPass\t\tErrors\t\tAverage\t\tNr. Samples\tSince Start")
    
    # 가중치 초기화
    if opts["random_init"]:
        random.seed(3003)
        weights = [random.random()] * opts["D"]
    else:
        weights = [0.] * opts["D"]
        
    # 학습 실행
    for pass_nr in range(opts["n_passes"]):
        error_counter = 0
        for e, (label, id, features) in enumerate( get_data_tsv(loc_dataset,opts) ):
            
            # 퍼셉트론은 지도학습 분류기의 일종이다. 
            # 이전 값에 대한 학습으로 예측을 한다. 
            # 내적(dotproduct) 값이 임계 값보다 높거나 낮은지에 따라 
            # 초과하면 "1"을 예측하고 미만이면 "0"을 예측한다.
            dp = dot_product(features, weights) > 0.5
            
            # 다음 perceptron은 샘플의 레이블을 본다. 
            # 실제 레이블 데이터에서 위 퍼셉트론으로 구한 dp값을 빼준다.
            # 예측이 정확하다면, error 값은 "0"이며, 가중치만 남겨 둔다. 
            # 예측이 틀린 경우 error 값은 "1" 또는 "-1"이고 다음과 같이 가중치를 업데이트 한다.
            # weights[feature_index] += learning_rate * error * feature_value
            error = label - dp
            
            # 예측이 틀리면 퍼셉트론은 가중치를 업데이트
            if error != 0:
                error_counter += 1
                # Updating the weights
                for index, value in features:
                    weights[index] += opts["learning_rate"] * error * log(1.+value)
                    
        #Reporting stuff
        print("%s\t\t%s\t\t%s\t\t%s\t\t%s" % ( \
            pass_nr+1,
            error_counter,
            round(1 - error_counter /float(e+1),5),
            e+1,datetime.now()-start))
        
        # error_counter가 0 이거나 오버피팅되는 경우면 중지
        if error_counter == 0 or error_counter < opts["errors_satisfied"]:
            print("%s errors found during training, halting"%error_counter)
            break
    
    return weights

In [19]:
def test_tron(loc_dataset, weights, opts):
    """
        output:
                preds: list, a list with
                [id,prediction,dotproduct,0-1normalized dotproduct]
    """
    start = datetime.now()
    print("\nTesting online\nErrors\t\tAverage\t\tNr. Samples\tSince Start")
    
    # 값 초기화
    preds = []
    error_counter = 0
    for e, (label, id, features) in enumerate( get_data_tsv(loc_dataset,opts) ):

        dotp = dot_product(features, weights)
        # 내적이 0.5보다 크다면 긍정으로 예측한다.
        dp = dotp > 0.5
        if dp > 0.5: # we predict positive class
            preds.append( [id, 1, dotp ] )
        else:
            preds.append( [id, 0, dotp ] )
    
        # get_data_tsv에서 테스트 데이터의 레이블을 1로 초기화해주었음
        if label - dp != 0:
            error_counter += 1
            
    print("%s\t\t%s\t\t%s\t\t%s" % (
        error_counter,
        round(1 - error_counter /float(e+1),5),
        e+1,
        datetime.now()-start))
    
    # 내적을 구해 0과 1로 일반화 한다.
    # TODO: proper probability (bounded sigmoid?), 
    # online normalization
    max_dotp = max(preds,key=itemgetter(2))[2]
    min_dotp = min(preds,key=itemgetter(2))[2]
    # 정규화된 값을 마지막에 추가
    # (피처와 가중치에 대한 내적값 - 최소 내적값) / 최대 내적값 - 최소 내적값
    for p in preds:
        p.append((p[2]-min_dotp)/float(max_dotp-min_dotp))
        
    #Reporting stuff
    print("Done testing in %s"%str(datetime.now()-start))
    return preds

In [20]:
# 옵션 세팅
opts = {}
opts["D"] = 2 ** 25
opts["learning_rate"] = 0.1
opts["n_passes"] = 80 # Maximum number of passes to run before halting (for 루프를 몇 번 돌건지)
opts["errors_satisfied"] = 0 # Halt when training errors < errors_satisfied (error가 0이어야만 멈춘다)
opts["random_init"] = False # set random weights, else set all 0 (랜덤값 사용여부)
opts["clean"] = True # clean the text a little (데이터 정제)
opts["2grams"] = True # add 2grams (바이그램 사용 여부)

In [23]:
#training and saving model into weights
%time weights = train_tron("word_tutorial/labeledTrainData.tsv",opts)


Pass		Errors		Average		Nr. Samples	Since Start
1		5690		0.7724		25000		0:00:13.213260
2		3087		0.87652		25000		0:00:24.311678
3		2188		0.91248		25000		0:00:35.446354
4		1601		0.93596		25000		0:00:48.621787
5		1277		0.94892		25000		0:00:59.910257
6		997		0.96012		25000		0:01:10.454385
7		886		0.96456		25000		0:01:21.052189
8		701		0.97196		25000		0:01:31.733266
9		512		0.97952		25000		0:01:42.290155
10		482		0.98072		25000		0:01:52.922136
11		333		0.98668		25000		0:02:03.431161
12		288		0.98848		25000		0:02:13.868452
13		242		0.99032		25000		0:02:24.414194
14		183		0.99268		25000		0:02:34.957586
15		196		0.99216		25000		0:02:45.507495
16		173		0.99308		25000		0:02:57.204046
17		145		0.9942		25000		0:03:08.289193
18		190		0.9924		25000		0:03:19.045930
19		110		0.9956		25000		0:03:29.716936
20		148		0.99408		25000		0:03:40.309897
21		117		0.99532		25000		0:03:50.939257
22		92		0.99632		25000		0:04:01.508224
23		103		0.99588		25000		0:04:12.109280
24		77		0.99692		25000		0:04:22.637463
25

In [25]:
# testing and saving predictions into preds
%time preds = test_tron("word_tutorial/testData.tsv",weights,opts)


Testing online
Errors		Average		Nr. Samples	Since Start
12290		0.5084		25000		0:00:09.715700
Done testing in 0:00:09.723679
Wall time: 9.72 s


In [26]:
preds[:10]

[['"12311_10"', 1, 74.92921021852996, 0.6381828805141688],
 ['"8348_2"', 0, -94.96116373671242, 0.4591732398480865],
 ['"5828_4"', 1, 6.584898215319436, 0.5661700262927256],
 ['"7186_2"', 0, -3.0498475944637784, 0.556018112766579],
 ['"12128_7"', 1, 35.14256205438916, 0.5962605901256207],
 ['"2913_8"', 1, 67.7897942587626, 0.6306602395559451],
 ['"4396_1"', 0, -33.68695297521327, 0.5237364884604149],
 ['"395_2"', 0, -11.159669607015141, 0.5474729769208296],
 ['"10616_1"', 0, -88.16832136722492, 0.4663307040607655],
 ['"9074_9"', 0, -28.280404966845783, 0.5294332456909144]]

### preds에서 4번째 있는 것이 정규화한 값이다
### 2번째는 판정한 결과값

In [27]:
import pandas as pd

presult = pd.DataFrame(preds)
presult.head(10)

,0,1,2,3
0,"""12311_10""",1,74.929210,0.638183
1,"""8348_2""",0,-94.961164,0.459173
2,"""5828_4""",1,6.584898,0.566170
3,"""7186_2""",0,-3.049848,0.556018
4,"""12128_7""",1,35.142562,0.596261
5,"""2913_8""",1,67.789794,0.630660
6,"""4396_1""",0,-33.686953,0.523736
7,"""395_2""",0,-11.159670,0.547473
8,"""10616_1""",0,-88.168321,0.466331
9,"""9074_9""",0,-28.280405,0.529433


In [28]:
output_sentiment = presult[1].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

-420


1    12710
0    12290
Name: 1, dtype: int64

단일 노드 단일 레이어 퍼셉트론은 비선형 함수를 모델링 할 수 없으므로 더 나은 NLP 출력을 얻으려면 FFN(feedforward nets), recurrent nets, self-organizing maps, MLP(Multi-layer Perceptrons), word2vec 및 extreme learning machine (백프로파게이션이 없는 fast ffnets)을 봐야할 것이다.